# Kaggle LLM Science Exam Perplexity Ranking Ensemble

This code is based on the discussion and notebooks from [Psi](https://www.kaggle.com/code/philippsinger) and [Takamichi Toda](https://www.kaggle.com/takamichitoda)

- https://www.kaggle.com/competitions/kaggle-llm-science-exam/discussion/424242
- https://www.kaggle.com/code/philippsinger/h2ogpt-perplexity-ranking
- https://www.kaggle.com/code/takamichitoda/llm-perplexity-ranking-ensemble/notebook

The idea is, rather than using or training an LLM to predict the correct answer given the question and multiple choice answers, we instead:
- Input each answer, prepended with the question, to a pretrained LLM and perform inference
- For each token, see what the model predicts as its logprob
- Calculate the perplexity over these results -- effectively treating the questions and multiple choice answers as a dataset and use the perplexity as the measure of how well the model is modelling the correct distribution that should correspond to the correct answer
- Sort the perplexity for each of the question-answer pairs to get the answer as that with the lowest perplexity and use that as the model's answer to the question

In [1]:
# !pip install bitsandbytes

In [2]:
import torch

import gc
import os
import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

/home/dom/projects/ai/llms/kaggle-science-exam/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from pathlib import Path

# TODO fix up for colab or kaggle if needed. No need for local or docker with volume mapped for HF cache
cwd = Path(os.getcwd())
data_dir =  cwd / 'data'
model_dir = [cwd / 'models']

In [4]:
import os
import subprocess
from pathlib import Path
import zipfile

COMPETITION='kaggle-llm-science-exam'

# fix as needed
# data_dir = Path(os.getcwd()) / 'data'
data_dir.mkdir(exist_ok=True)

file_path = data_dir / f'{COMPETITION}.zip'

if not os.path.exists(file_path):
    # download dataset
    subprocess.run(['kaggle', 'competitions', 'download', '-p', data_dir, '-c', COMPETITION], check=True)
    # subprocess.run(['unzip', 'kaggle-llm-science-exam.zip'], check=True)
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)

In [5]:
MODELS = ["mistralai/Mistral-7B-v0.1",
          "01-ai/Yi-6B"]

In [21]:
train_df = pd.read_csv(data_dir / 'train.csv')

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    test_df = pd.read_csv(data_dir / 'test.csv', index_col='id')
    # add unused answer to avoid errors when we're using the train set as test set
    test_df["answer"] = "A"
else:
    test_df = train_df.copy()
test_df.head()

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [60]:
# https://www.kaggle.com/code/philippsinger/h2ogpt-perplexity-ranking/notebook

class Perplexity(nn.Module):
    def __init__(self, reduce: bool = True):
        super().__init__()
        self.reduce = reduce
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, logits, labels):
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()

        perplexity = []
        for i in range(labels.shape[0]):
            perplexity.append(self.loss_fn(shift_logits[i], shift_labels[i]))
        perplexity = torch.stack(perplexity, dim=0)
        # perplexity = torch.exp(perplexity)
        if self.reduce:
            perplexity = perplexity.mean()
        return perplexity

In [61]:
# https://www.kaggle.com/code/philippsinger/h2ogpt-perplexity-ranking/notebook
def precision_at_k(r, k):
    """Precision at k"""
    assert k <= len(r)
    assert k != 0
    return sum(int(x) for x in r[:k]) / k

def MAP_at_3(predictions, true_items):
    """Score is mean average precision at 3"""
    U = len(predictions)
    map_at_3 = 0.0
    for u in range(U):
        user_preds = predictions[u]
        user_true = true_items[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]
        for k in range(min(len(user_preds), 3)):
            map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
    return map_at_3 / U

# NB the calculation here is missing the short circuiting of the precision_at_k function
# This would be needed if the model output multiple copies of the same answer
# This is specced in the MAP@3 calculation description in the competition rules
# https://www.kaggle.com/competitions/kaggle-llm-science-exam
# redo it here for clarity
# The code that invokes this function won't let this happen as it uses [cols][np.argsort(preplexity)]
# which won't return the same index for tied values. Still though, this feels like a bug in the calc so
# worth noting
# TODO test this
def MAP_at_3(predictions, true_items):
    """Score is mean average precision at 3"""
    U = len(predictions)
    map_at_3 = 0.0
    for u in range(U):
        user_preds = predictions[u]
        user_true = true_items[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]
        for k in range(min(len(user_preds), 3)):
            map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
            if user_results[k] == 1:
                break
    return map_at_3 / U

In [62]:
perplexity = Perplexity()
candidate = np.array(['A', 'B', 'C', 'D', 'E'])

In [26]:
# https://www.kaggle.com/code/takamichitoda/llm-perplexity-ranking-ensemble
def infer(model_name, infer_df):
    # tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        load_in_8bit=True,
        device_map="auto",
        trust_remote_code=True,
    )

    perps, scores = [], []
    for _, row in tqdm(infer_df.iterrows(), total=len(infer_df)):
        inp = row["prompt"]
        cands = [f"Question: {inp}\n Answer: {row[c]}" for c in candidate]

        with torch.no_grad():
            inputs = tokenizer(cands, return_tensors="pt", padding=True, truncation=True).to(f"cuda:{model.device.index}")
            output = model(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask,)
            output = output.logits
            labels = inputs.input_ids
            labels.masked_fill_(~inputs["attention_mask"].bool(), -100)
            perp = [perplexity(output[i].unsqueeze(0), labels[i].unsqueeze(0)).cpu() for i in range(len(cands))]

        # save perplexity not the sorted predictions for each question so we can use it for ensembling
        perps.append(perp)

        # calculate the MAP@3 score for each question, MAP_at_3 expects a list of lists and a list of answers
        # but we're doing them individually here so we need to wrap them in lists
        preds = candidate[np.argsort(np.asarray(perp))]
        score = MAP_at_3([preds], [row["answer"]])
        scores.append(score)

        torch.cuda.empty_cache()
        gc.collect()

    return perps, scores


In [85]:
# TODO delete
perplexity(torch.tensor([[[0.1, 0.1, 0.1], [0.1, 0.1, 0.1]]]), torch.tensor([[0, 0]])).cpu()

tensor(1.0986)

In [ ]:
model_perps = []
for model_name in MODELS:
    perps, scores = infer(model_name, test_df)
    model_perps.append(perps)
    torch.cuda.empty_cache()
    gc.collect()

    print(f"{model_name} MAP@3: {np.mean(scores)}")

perps = np.array(model_perps)

In [ ]:
perps_avg = perps[0] if len(perps) == 1 else np.mean(perps, axis=0)

In [ ]:
sub_df = pd.read_csv(data_dir / 'sample_submission.csv')
sub_df["prediction"] = [" ".join(candidate[np.argsort(perp)][:3]) for perp in perps_avg]

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv(data_dir / 'submission.csv', index=False)

In [ ]:
pd.read_csv(data_dir / "submission.csv")